In the FSM1 workbook, I had thought that Surprise needed a different input. After reviewing the Suprise documentation, we can almost run it off of our original data. We'll only need to do a little bit of cleaning.

In future iterations, I'll use the amount of times a song is played as a sort of explicit rating. For now, we'll just apply a score of '3' if a song was played for an account.

In [ ]:
# !pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 3.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1634009 sha256=2db29feb5d7218562c860da9f7216aee0644bd1d6ad48725f861ceac819cc7df
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds


from google.cloud import bigquery

from surprise import Reader, Dataset
from surprise.prediction_algorithms import SVD
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, SVDpp

In [ ]:
# Authenticate google cloud platform
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

Authenticated


In [ ]:
project_id = 'listenbrainzrecommender'
client = bigquery.Client(project=project_id)

In [ ]:
# get user-song plays
plays_1000 = client.query('''
SELECT
  user_name, artist_name, track_name
FROM
  `listenbrainz.listenbrainz.listen`
GROUP BY
  user_name, artist_name, track_name
LIMIT
  1000
''').to_dataframe()

In [ ]:
plays_1000.head()

,user_name,artist_name,track_name
0,deathwarmedover,Great Grandpa,Teen Challenge
1,deathwarmedover,Stealing Sheep,Shut Eye
2,deathwarmedover,Sufjan Stevens,John My Beloved
3,deathwarmedover,Tegan and Sara,You Went Away
4,deathwarmedover,Crystal Castles,Courtship Dating


In [ ]:
# create df for only songs, to create unique id's for songs
# unique id will be index
song_df = plays_1000[['artist_name', 'track_name']]
song_df.head()

,artist_name,track_name
0,Great Grandpa,Teen Challenge
1,Stealing Sheep,Shut Eye
2,Sufjan Stevens,John My Beloved
3,Tegan and Sara,You Went Away
4,Crystal Castles,Courtship Dating


In [ ]:
# drop duplicate songs
song_df_cl = song_df.drop_duplicates()
song_df_cl.head()

,artist_name,track_name
0,Great Grandpa,Teen Challenge
1,Stealing Sheep,Shut Eye
2,Sufjan Stevens,John My Beloved
3,Tegan and Sara,You Went Away
4,Crystal Castles,Courtship Dating


In [ ]:
def get_song_id(s):
  try:
    return song_df_cl[(song_df_cl['artist_name'].eq(s['artist_name'])) &
                       (song_df_cl['track_name'].eq(s['track_name']))].index.item()
  except IndexError:
    pass

In [ ]:
# *a = tuple(zip(plays_1000["artist_name"], plays_1000["track_name"]))

In [ ]:
# plays_1000['song_id'] = song_df_cl.iloc[np.where((plays_1000["artist_name"] == song_df_cl["artist_name"]) & \
#                                                              (plays_1000["track_name"] == song_df_cl["track_name"]))]

In [ ]:
plays_1000['song_id'] = plays_1000.apply(get_song_id, axis=1)

KeyError: ignored

In [ ]:
plays_1000.drop(columns=['artist_name','track_name'], inplace=True)
plays_1000['song_score'] = 3

In [ ]:
plays_1000.head()

,user_name,song_id,song_score
0,deathwarmedover,0,3
1,deathwarmedover,1,3
2,deathwarmedover,2,3
3,deathwarmedover,3,3
4,deathwarmedover,4,3


In [ ]:
# v3 song class
class Song:
  def __init__(self, artist_name, song_name):
    self.artist_name = artist_name
    self.song_name = song_name
    self.song_id = id_no
    id_no += 1
    
  
  def get_listener_list(self, request):
    '''
    Gets the listeners for a given song.
    Does not take into account the amount an account has listened to a song.
    Returns a dataframe if requested.
    '''
    self.listeners = client.query('''
      SELECT
        user_name
      FROM
        `listenbrainz.listenbrainz.listen`
      WHERE
        artist_name = \''''
      + self.artist_name + '''\'
        AND
        track_name = \''''
      + self.song_name + '''\'
      GROUP BY
        user_name
      ''').to_dataframe()

    if request:
      return self.listeners

    

In [ ]:
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(plays_1000, reader)

In [ ]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  57 

Number of items:  999


In [ ]:
# FSM will use KNNBaseline with default hyperparameters
knn_baseline = KNNBaseline()
cv_knn_baseline = cross_validate(knn_baseline, data, n_jobs=-1)

In [ ]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0., 0., 0., 0., 0.]))
('test_mae', array([0., 0., 0., 0., 0.]))
('fit_time', (0.049164533615112305, 0.05620121955871582, 0.05515313148498535, 0.04967498779296875, 0.027332782745361328))
('test_time', (0.042463064193725586, 0.021831989288330078, 0.0232696533203125, 0.021255970001220703, 0.01289820671081543))
-----------------------


0.0

It doesn't look like we have enough data for accounts to have any of the same songs listened. We'll try again with a larger dataset.

In [ ]:
# get user-song plays
plays_10000 = client.query('''
SELECT
  user_name, artist_name, track_name
FROM
  `listenbrainz.listenbrainz.listen`
GROUP BY
  user_name, artist_name, track_name
LIMIT
  10000
''').to_dataframe()

In [ ]:
# create df for only songs, to create unique id's for songs
# unique id will be index
song_df = plays_10000[['artist_name', 'track_name']]
song_df.head()

# drop duplicate songs
song_df_cl = song_df.drop_duplicates()
song_df_cl.head()

plays_10000['song_id'] = plays_10000.apply(get_song_id, axis=1)

plays_10000.drop(columns=['artist_name','track_name'], inplace=True)
plays_10000['song_score'] = 3

In [ ]:
plays_10000.song_id.value_counts()

2228    3
1897    3
2473    3
1071    3
1845    3
       ..
3350    1
3351    1
3352    1
3353    1
9999    1
Name: song_id, Length: 9896, dtype: int64

In [ ]:
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(plays_10000, reader)

dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  422 

Number of items:  9896


In [ ]:
# FSM will use KNNBaseline with default hyperparameters
knn_baseline = KNNBaseline()
cv_knn_baseline = cross_validate(knn_baseline, data, n_jobs=-1)

In [ ]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0., 0., 0., 0., 0.]))
('test_mae', array([0., 0., 0., 0., 0.]))
('fit_time', (0.05614924430847168, 0.04935884475708008, 0.05117201805114746, 0.05314278602600098, 0.026962995529174805))
('test_time', (0.024069547653198242, 0.022206544876098633, 0.02367544174194336, 0.022855043411254883, 0.012890815734863281))
-----------------------


0.0

We can see that there are still just too many options for the system to recommend anything. Even after getting 10k samples. There are very few repeated songs that makes it hard for the model to predict any songs. There is likely also an issue with having all songs played at a score of '3'.

We can also use SVD++, which is the only surprise algorithm that takes into account an implicit rating.

In [ ]:
# Trying SVD++ with default hyperparameters
svdpp = SVDpp()
cv_svdpp = cross_validate(svdpp, data, n_jobs=-1)

In [ ]:
# print out the average score for the test set
for i in cv_svdpp.items():
    print(i)
print('-----------------------')
np.mean(cv_svdpp['test_rmse'])

('test_rmse', array([0.01054373, 0.01113369, 0.00982714, 0.01030544, 0.01044496]))
('test_mae', array([0.00595371, 0.0059411 , 0.00621597, 0.00573801, 0.00589638]))
('fit_time', (19.751715421676636, 20.26791548728943, 20.14812707901001, 19.480408430099487, 9.574454545974731))
('test_time', (0.03039860725402832, 0.026179075241088867, 0.02298760414123535, 0.02765512466430664, 0.03253483772277832))
-----------------------


0.010450992449566437

Nice! Now we are getting somewhere! We may have only scored 0.01 but at least it's something!!

We have lots of room to improve now. Next we'll be moving on to gridsearching and honing our methods/pipelines for further testing.